In [ ]:
!pip install -q eccd_datasets pygradus

In [ ]:
import io
import json
from PIL import Image
from eccd_datasets import load_images

import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Configuración inicial y carga de etiquetas
torch.manual_seed(42)
!wget https://files.fast.ai/models/imagenet_class_index.json -O resnet_labels.json
with open("resnet_labels.json", "r") as fh:
    data = json.load(fh)
resnet_labels = {int(k): v[1] for k, v in data.items()}

--2023-11-24 12:47:35--  https://files.fast.ai/models/imagenet_class_index.json
Resolving files.fast.ai (files.fast.ai)... 104.26.3.19, 172.67.69.159, 104.26.2.19, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35363 (35K) [application/json]
Saving to: ‘resnet_labels.json’

resnet_labels.json  100%[===================>]  34.53K  --.-KB/s    in 0.01s   

2023-11-24 12:47:35 (2.32 MB/s) - ‘resnet_labels.json’ saved [35363/35363]



In [ ]:
# Carga de imágenes desde DataFrame
df_images = load_images()
df_images.head()

,dataset,coarse_cat,finegrained_cat,image_data
0,train,Apple,Golden-Delicious,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,train,Apple,Golden-Delicious,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,train,Apple,Golden-Delicious,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,train,Apple,Golden-Delicious,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,train,Apple,Golden-Delicious,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


In [ ]:
# Crear un mapeo de categorías a índices numéricos
label_map = {category: idx for idx, category in enumerate(df_images['coarse_cat'].unique())}

In [ ]:
# Definición de la clase personalizada para Dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, label_map, transform=None):
        self.dataframe = dataframe
        self.label_map = label_map
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_data = self.dataframe.iloc[idx, 3]
        image = Image.open(io.BytesIO(image_data))
        label = self.label_map[self.dataframe.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

In [ ]:
# Transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
# Crear conjuntos de datos
train_data = df_images[df_images['dataset'] == 'train']
val_data = df_images[df_images['dataset'] != 'train']

In [ ]:
train_dataset = CustomDataset(dataframe=train_data, label_map=label_map, transform=transform)
val_dataset = CustomDataset(dataframe=val_data, label_map=label_map, transform=transform)

In [ ]:
# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
# Cargar el modelo ResNet preentrenado y modificarlo
resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

In [ ]:
# Congelar todas las capas excepto la última
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
# Modificar la última capa del ResNet para el número de clases específico
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, len(label_map))

In [ ]:
# Preparar el modelo para el entrenamiento
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.AdamW(resnet.fc.parameters(), lr=0.001)
optimizer = optim.RMSprop(resnet.fc.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        # Iterar sobre los datos de entrenamiento
        for inputs, labels in train_loader:
            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward + optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

        # Evaluar el modelo en el conjunto de validación
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Acc: {val_acc:.4f}')
        scheduler.step(val_loss)

In [ ]:
# Entrenar el modelo
train_model(resnet, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=32)

Epoch 0/31, Loss: 0.9230, Acc: 0.9259
Validation Loss: 4.2766, Acc: 0.7745
Epoch 1/31, Loss: 0.4072, Acc: 0.9541
Validation Loss: 3.1312, Acc: 0.8071
Epoch 2/31, Loss: 0.4464, Acc: 0.9585
Validation Loss: 2.1995, Acc: 0.8165
Epoch 3/31, Loss: 0.2461, Acc: 0.9659
Validation Loss: 2.9941, Acc: 0.8071
Epoch 4/31, Loss: 0.4352, Acc: 0.9363
Validation Loss: 3.1679, Acc: 0.7636
Epoch 5/31, Loss: 0.1512, Acc: 0.9719
Validation Loss: 2.3260, Acc: 0.8078
Epoch 6/31, Loss: 0.1014, Acc: 0.9748
Validation Loss: 2.6063, Acc: 0.8093
Epoch 7/31, Loss: 0.1417, Acc: 0.9807
Validation Loss: 2.0061, Acc: 0.8180
Epoch 8/31, Loss: 0.1224, Acc: 0.9748
Validation Loss: 3.8411, Acc: 0.7687
Epoch 9/31, Loss: 0.1682, Acc: 0.9689
Validation Loss: 2.7106, Acc: 0.7912
Epoch 10/31, Loss: 0.1463, Acc: 0.9793
Validation Loss: 2.0486, Acc: 0.8231
Epoch 11/31, Loss: 0.4793, Acc: 0.9333
Validation Loss: 3.5959, Acc: 0.7788
Epoch 12/31, Loss: 0.2751, Acc: 0.9630
Validation Loss: 2.6743, Acc: 0.8209
Epoch 13/31, Loss: 0.2

--------------------------------------------------------------------------

In [ ]:
def obtener_predicciones(modelo, data_loader):
    modelo.eval()
    predicciones = []
    etiquetas_reales = []
    with torch.no_grad():
        for imagenes, etiquetas in data_loader:
            outputs = modelo(imagenes)
            _, preds = torch.max(outputs, 1)
            predicciones.extend(preds.tolist())
            etiquetas_reales.extend(etiquetas.tolist())
    return predicciones, etiquetas_reales

predicciones, etiquetas_reales = obtener_predicciones(resnet, val_loader)


In [ ]:
nombre_categorias = {v: k for k, v in label_map.items()}

predicciones_nombres = [nombre_categorias[i] for i in predicciones]
etiquetas_reales_nombres = [nombre_categorias[i] for i in etiquetas_reales]


In [ ]:
def calcular_costo_y_contar(predicho, real, precios):
    costo = 0
    cobro_de_mas = 0
    cobro_de_menos = 0
    aciertos = 0

    if predicho == real:
        aciertos = 1
    elif precios[predicho] > precios[real]:
        costo = 5 * (precios[predicho] - precios[real])
        cobro_de_mas = 1
    else:
        costo = precios[real] - precios[predicho]
        cobro_de_menos = 1

    return costo, cobro_de_mas, cobro_de_menos, aciertos

In [ ]:
# Precios por categoría
precios = {
    "Apple": 74,
    "Pear": 89,
    "Satsumas": 44,
    "Melon": 109,
    "Orange": 35
}

In [ ]:
# Calcular el costo total y contar errores y aciertos
costo_total = 0
total_cobro_de_mas = 0
total_cobro_de_menos = 0
total_aciertos = 0

for real, predicho in zip(etiquetas_reales_nombres, predicciones_nombres):
    costo, cobro_de_mas, cobro_de_menos, aciertos = calcular_costo_y_contar(predicho, real, precios)
    costo_total += costo
    total_cobro_de_mas += cobro_de_mas
    total_cobro_de_menos += cobro_de_menos
    total_aciertos += aciertos

print("Costo total de errores:", costo_total)
print("Total de veces cobrando de más:", total_cobro_de_mas)
print("Total de veces cobrando de menos:", total_cobro_de_menos)
print("Total de aciertos:", total_aciertos)

Costo total de errores: 9315
Total de veces cobrando de más: 70
Total de veces cobrando de menos: 184
Total de aciertos: 1125


In [ ]:
def calcular_aciertos_y_pruebas_por_categoria(predicciones, etiquetas_reales):
    # Crear diccionarios para contar aciertos y pruebas por categoría
    aciertos_por_categoria = {categoria: 0 for categoria in nombre_categorias.values()}
    pruebas_por_categoria = {categoria: 0 for categoria in nombre_categorias.values()}

    for pred, real in zip(predicciones, etiquetas_reales):
        pruebas_por_categoria[real] += 1
        if pred == real:
            aciertos_por_categoria[pred] += 1

    return aciertos_por_categoria, pruebas_por_categoria

# Contar los aciertos y pruebas por categoría
aciertos_por_categoria, pruebas_por_categoria = calcular_aciertos_y_pruebas_por_categoria(predicciones_nombres, etiquetas_reales_nombres)

# Mostrar los aciertos, pruebas y porcentaje de aciertos por categoría
for categoria in nombre_categorias.values():
    aciertos = aciertos_por_categoria[categoria]
    pruebas = pruebas_por_categoria[categoria]
    porcentaje_aciertos = (aciertos / pruebas * 100) if pruebas > 0 else 0
    print(f"Categoría {categoria}: {aciertos} aciertos de {pruebas} pruebas ({porcentaje_aciertos:.2f}% de aciertos)")


Categoría Apple: 536 aciertos de 574 pruebas (93.38% de aciertos)
Categoría Pear: 172 aciertos de 221 pruebas (77.83% de aciertos)
Categoría Satsumas: 56 aciertos de 141 pruebas (39.72% de aciertos)
Categoría Melon: 271 aciertos de 326 pruebas (83.13% de aciertos)
Categoría Orange: 90 aciertos de 117 pruebas (76.92% de aciertos)
